
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
 !pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-26 23:36:00.454822: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 23:36:03.453589: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 23:36:03.453694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 23:36:03.910812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 23:36:04.842663: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 23:36:04.847189: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[15, 7], [30, 15], [30, 15, 7]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  3.29353501699984
Stopped at epoch:  55
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.5974633910000193
Stopped at epoch:  17
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  2.3041792080000505
Stopped at epoch:  67
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.25676947399983874
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.22406343300008302
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.3900872879999042
Stopped at epoch:  10
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 ++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_0_0,0.519350,-1.099948,0.484893,0.173215,57.966541,86.257202,58.872025,73.370056
model_5_0_1,0.599155,-0.459107,0.490335,0.333551,48.342010,59.934116,58.250011,59.141598
model_5_0_2,0.641252,-0.758347,0.412811,0.213238,43.265118,72.225632,67.110291,69.818413
model_5_0_3,0.659177,-0.783704,0.420869,0.211908,41.103306,73.267197,66.189285,69.936401
model_5_0_4,0.686096,-0.919010,0.432402,0.185741,37.856873,78.825043,64.871201,72.258492
model_5_0_5,0.689886,-1.102938,0.436083,0.142900,37.399841,86.380028,64.450485,76.060242
model_5_0_6,0.699080,-1.159435,0.438198,0.130339,36.291080,88.700676,64.208740,77.174988
model_5_0_23,0.705955,-1.454081,0.410595,0.041405,35.461887,100.803543,67.363510,85.067062
model_5_0_22,0.706088,-1.448589,0.410457,0.042668,35.445889,100.577934,67.379265,84.954987
model_5_0_21,0.707496,-1.422695,0.410711,0.049167,35.276073,99.514313,67.350288,84.378296


DataFrame salvo em ../results/metrics_5_0
+++++++++++ [5_1] | 1 ++++++++++++++++++
Elapsed time:  0.2438278510001055
Stopped at epoch:  6
1/1 [==============================] - 0s 35ms/step
+++++++++++ [5_1] | 2 ++++++++++++++++++
Elapsed time:  0.40815638399999443
Stopped at epoch:  8
1/1 [==============================] - 0s 20ms/step
+++++++++++ [5_1] | 3 ++++++++++++++++++
Elapsed time:  0.4308162210002138
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [5_1] | 4 ++++++++++++++++++
Elapsed time:  0.5723535959998571
Stopped at epoch:  8
1/1 [==============================] - 0s 21ms/step
+++++++++++ [5_1] | 5 ++++++++++++++++++
Elapsed time:  0.3088709880000806
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [5_1] | 6 ++++++++++++++++++
Elapsed time:  0.27332075799995437
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_1] | 7 ++++++++++++++++++
Elapsed time:  0.2206441209

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_5_1_9,0.516154,0.536173,-0.870205,-0.254855,58.351925,30.041603,271.711121,143.768692
model_5_1_8,0.525551,0.533624,-0.806755,-0.217750,57.218681,30.206717,262.492828,139.517578
model_5_1_7,0.610616,0.493635,-0.247467,0.104027,46.959816,32.796761,181.237137,102.651604
model_5_1_6,0.707530,0.437360,0.391026,0.468203,35.271946,36.441620,88.474258,60.928013
model_5_1_0,0.708642,0.379391,0.650506,0.605700,35.137890,40.196232,50.775978,45.174953
model_5_1_5,0.711220,0.437789,0.421506,0.486525,34.826897,36.413834,84.046021,58.828808
model_5_1_4,0.722887,0.374306,0.531148,0.532952,33.419849,40.525608,68.116753,53.509716
model_5_1_1,0.725154,0.446619,0.612227,0.602978,33.146553,35.841934,56.337223,45.486782
model_5_1_3,0.726001,0.384468,0.558894,0.552551,33.044312,39.867382,64.085709,51.264259
model_5_1_2,0.727636,0.480002,0.586164,0.597416,32.847137,33.679768,60.123756,46.123993


DataFrame salvo em ../results/metrics_5_1
+++++++++++ [5_2] | 1 ++++++++++++++++++
Elapsed time:  0.28300507599988123
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_2] | 2 ++++++++++++++++++
Elapsed time:  0.2202880610000193
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_2] | 3 ++++++++++++++++++
Elapsed time:  0.5984060030000364
Stopped at epoch:  16
1/1 [==============================] - 0s 15ms/step
+++++++++++ [5_2] | 4 ++++++++++++++++++
Elapsed time:  0.9814158500000758
Stopped at epoch:  28
1/1 [==============================] - 0s 19ms/step
+++++++++++ [5_2] | 5 ++++++++++++++++++
Elapsed time:  0.46106257200017353
Stopped at epoch:  9
1/1 [==============================] - 0s 23ms/step
+++++++++++ [5_2] | 6 ++++++++++++++++++
Elapsed time:  0.339393728999994
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [5_2] | 7 ++++++++++++++++++
Elapsed time:  0.389968861

...

DataFrame salvo em ../results/metrics_7_7
+++++++++++ [7_8] | 1 ++++++++++++++++++
Elapsed time:  0.2647146760000396
Stopped at epoch:  10
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_8] | 2 ++++++++++++++++++
Elapsed time:  0.16455360899999505
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_8] | 3 ++++++++++++++++++
Elapsed time:  0.3236088890002975
Stopped at epoch:  13
1/1 [==============================] - 0s 15ms/step
+++++++++++ [7_8] | 4 ++++++++++++++++++
Elapsed time:  0.18462790600005974
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [7_8] | 5 ++++++++++++++++++
Elapsed time:  1.7016258130001916
Stopped at epoch:  54
1/1 [==============================] - 0s 15ms/step
+++++++++++ [7_8] | 6 ++++++++++++++++++
Elapsed time:  3.548597467000036
Stopped at epoch:  127
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_8] | 7 ++++++++++++++++++
Elapsed time:  0.742618

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_7_8_0,0.722773,0.631075,0.170130,0.399711,33.433701,56.839142,148.494446,99.971092
model_7_8_13,0.775570,0.512488,0.413400,0.464635,27.066275,75.109451,104.964417,89.158813
model_7_8_12,0.775579,0.512538,0.413394,0.464657,27.065182,75.101746,104.965385,89.155197
model_7_8_11,0.775590,0.512597,0.413388,0.464682,27.063877,75.092743,104.966469,89.150932
model_7_8_17,0.775597,0.511614,0.414365,0.464694,27.063040,75.244110,104.791779,89.148872
model_7_8_16,0.775603,0.511669,0.414342,0.464709,27.062319,75.235725,104.795914,89.146408
model_7_8_14,0.775609,0.511696,0.414341,0.464722,27.061647,75.231544,104.796066,89.144234
model_7_8_18,0.775610,0.511599,0.414439,0.464724,27.061552,75.246429,104.778450,89.143890
model_7_8_15,0.775610,0.511703,0.414340,0.464725,27.061508,75.230537,104.796257,89.143829
model_7_8_19,0.775610,0.511601,0.414440,0.464726,27.061441,75.246162,104.778290,89.143593


DataFrame salvo em ../results/metrics_7_8
+++++++++++ [7_9] | 1 ++++++++++++++++++
Elapsed time:  0.3320546110003306
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [7_9] | 2 ++++++++++++++++++
Elapsed time:  0.3762688669999079
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_9] | 3 ++++++++++++++++++
Elapsed time:  0.1662207969998235
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_9] | 4 ++++++++++++++++++
Elapsed time:  0.2326813790000415
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [7_9] | 5 ++++++++++++++++++
Elapsed time:  0.16929069300022093
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [7_9] | 6 ++++++++++++++++++
Elapsed time:  0.17904871200016714
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [7_9] | 7 ++++++++++++++++++
Elapsed time:  0.1675300269

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_7_9_0,0.765038,0.731651,0.961833,0.819876,28.336430,26.353868,1.930031,14.860311
model_7_9_1,0.777411,0.770655,0.955643,0.842672,26.844301,22.523359,2.243077,12.979659
model_7_9_2,0.784985,0.776193,0.946371,0.843488,25.930887,21.979448,2.711898,12.912350
model_7_9_3,0.789454,0.765480,0.932569,0.832755,25.391928,23.031559,3.409838,13.797800
model_7_9_4,0.794855,0.757490,0.923342,0.825058,24.740561,23.816223,3.876467,14.432808
model_7_9_5,0.799223,0.742567,0.915982,0.813531,24.213770,25.281780,4.248648,15.383838
model_7_9_6,0.803827,0.731770,0.912885,0.805833,23.658527,26.342155,4.405256,16.018913
model_7_9_7,0.806992,0.717660,0.911074,0.796418,23.276834,27.727884,4.496843,16.795639
model_7_9_8,0.811481,0.714292,0.909897,0.793957,22.735479,28.058577,4.556345,16.998705
model_7_9_9,0.815211,0.701350,0.906389,0.784788,22.285620,29.329641,4.733760,17.755104


DataFrame salvo em ../results/metrics_7_9
Testando combinacao8: Hidden Size=[30, 15], regularizer=0.1, learning_rate=0.1
+++++++++++ [8_0] | 1 ++++++++++++++++++
Elapsed time:  0.6292953020001733
Stopped at epoch:  12
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_0] | 2 ++++++++++++++++++
Elapsed time:  3.5961382220002633
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_0] | 3 ++++++++++++++++++
Elapsed time:  0.026743374000034237
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_0] | 4 ++++++++++++++++++
Elapsed time:  0.027355806000286975
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_0] | 5 ++++++++++++++++++
Elapsed time:  0.02897159800022564
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_0] | 6 ++++++++++++++++++
Elapsed time:  0.027502644000378496
Stopped at epoch:  0
1/1 [==============================